# Prerequisites
For usage from within Docker container:
1. Start docker container on host with argument `-p 8888:8888`
2. From within docker:
    - Install jupyter notebook: `pip install notebook`
    - Start jupyter notebook: `jupyter notebook --ip 0.0.0.0 --no-browser --allow-root`
3. On the host visit `localhost:8888/tree`

# Rendered images

## Necessary Imports

In [1]:
import os
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
verbosity = 2
if verbosity > 0:
    print(f"Torch {torch.__version__}")
    if torch.cuda.is_available():
        print("CUDA available")
    else:
        print("CUDA unavailable")
    print(f"Device: {device}")
import sys
sys.path.append('/media/kristijan/kristijan-hdd-ex/ShapeFromImages/')

from data.on_the_fly_smpl_train_dataset import OnTheFlySMPLTrainDataset
from configs import paths
from configs.poseMF_shapeGaussian_net_config import get_poseMF_shapeGaussian_cfg_defaults

/media/kristijan/kristijan-hdd-ex/ShapeFromImages/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch 1.4.0
CUDA available
Device: cuda:0


In [2]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

In [3]:
from renderers.pytorch3d_textured_renderer import TexturedIUVRenderer
import pytorch3d
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex, TexturesUV
from pytorch3d.io import IO
if verbosity > 0:
    print(f"PyTorch3D {pytorch3d.__version__}")
from models.smpl_official import SMPL
from smplx.lbs import batch_rodrigues
from utils.augmentation.smpl_augmentation import normal_sample_shape
from utils.augmentation.cam_augmentation import augment_cam_t
from utils.augmentation.rgb_augmentation import augment_rgb
from utils.augmentation.lighting_augmentation import augment_light
from utils.rigid_transform_utils import aa_rotate_rotmats_pytorch3d, aa_rotate_translate_points_pytorch3d
import numpy as np
import matplotlib.pyplot as plt

ImportError: libtorch_cpu.so: cannot open shared object file: No such file or directory

## Load Configuration

In [ ]:
pose_shape_cfg = get_poseMF_shapeGaussian_cfg_defaults()
if verbosity > 1:
    print(f"Configuration: \n{pose_shape_cfg}")

## Some Default Tensors

In [ ]:
x_axis = torch.tensor([1., 0., 0.], device=device, dtype=torch.float32)
delta_betas_std_vector = torch.ones(
    pose_shape_cfg.MODEL.NUM_SMPL_BETAS,
    device=device,
    dtype=torch.float32
) * pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.SMPL.SHAPE_STD
mean_shape = torch.zeros(
    pose_shape_cfg.MODEL.NUM_SMPL_BETAS,
    device=device,
    dtype=torch.float32
)
mean_cam_t = torch.tensor(
    pose_shape_cfg.TRAIN.SYNTH_DATA.MEAN_CAM_T,
    device=device,
    dtype=torch.float32
)
mean_cam_t = mean_cam_t[None, :].expand(pose_shape_cfg.TRAIN.BATCH_SIZE, -1)

## Load Train and Validation Datasets

In [ ]:
train_dataset = OnTheFlySMPLTrainDataset(
    poses_path=paths.TRAIN_POSES_PATH,
    textures_path=paths.TRAIN_TEXTURES_PATH,
    backgrounds_dir_path=paths.TRAIN_BACKGROUNDS_PATH,
    params_from='not_amass',
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)

val_dataset = OnTheFlySMPLTrainDataset(
    poses_path=paths.VAL_POSES_PATH,
    textures_path=paths.VAL_TEXTURES_PATH,
    backgrounds_dir_path=paths.VAL_BACKGROUNDS_PATH,
    params_from='all',
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)
if verbosity > 0:
    print("Training:")
    print(f"    Poses found: {len(train_dataset)}")
    print(f"    Textures found (gray / non-gray): {len(train_dataset.grey_textures)} / {len(train_dataset.nongrey_textures)}")
    print(f"    Backgrounds found: {len(train_dataset.backgrounds_paths)}")
    print("Validation:")
    print(f"    Poses found: {len(val_dataset)}")
    print(f"    Textures found (gray / non-gray): {len(val_dataset.grey_textures)} / {len(val_dataset.nongrey_textures)}")
    print(f"    Backgrounds found: {len(val_dataset.backgrounds_paths)}")

## Train and Validation Data Loaders

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=pose_shape_cfg.TRAIN.NUM_WORKERS,
    pin_memory=pose_shape_cfg.TRAIN.PIN_MEMORY
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=pose_shape_cfg.TRAIN.NUM_WORKERS,
    pin_memory=pose_shape_cfg.TRAIN.PIN_MEMORY
)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}

## Initialize PyTorch3D Renderer

In [ ]:
pytorch3d_renderer = TexturedIUVRenderer(
    device=device,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE,
    projection_type='perspective',
    perspective_focal_length=pose_shape_cfg.TRAIN.SYNTH_DATA.FOCAL_LENGTH,
    render_rgb=True,
    bin_size=32
)

NameError: name 'TexturedIUVRenderer' is not defined

## Initialize SMPL Model

In [ ]:
smpl_model = SMPL(paths.SMPL, num_betas=pose_shape_cfg.MODEL.NUM_SMPL_BETAS).to(device)

## Fetch Training Data

In [ ]:
for batch in train_dataloader:
    target_poses = batch['pose'].to(device)
    backgrounds = batch['background'].to(device)
    textures = batch['texture'].to(device)
    if verbosity > 0:
        print(f"Poses shape: {target_poses.shape}")
        print(f"Backgrounds shape: {backgrounds.shape}")
        print(f"Textures shape: {textures.shape}")
    break
# Randomly sample body shape
target_shapes = normal_sample_shape(
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    mean_shape=mean_shape,
    std_vector=delta_betas_std_vector
)
# Randomly sample camera translation
target_cam_t = augment_cam_t(
    mean_cam_t,
    xy_std=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.CAM.XY_STD,
    delta_z_range=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.CAM.DELTA_Z_RANGE
)
if verbosity > 0:
    print(f"Shapes shape: {target_shapes.shape}")
    print(f"Camera translation shape: {target_cam_t.shape}")

## Visualize the Batch

In [ ]:
if verbosity > 1:
    batch_size = batch['pose'].shape[0]
    fig, axs = plt.subplots(batch_size, 2, figsize=(10,20))
    for idx in range(batch_size):
        axs[idx, 0].imshow(backgrounds[idx].permute(1,2,0).cpu())
        axs[idx, 1].imshow(textures[idx].cpu())
        axs[idx, 0].set_title(f"Background Sample {idx}")
        axs[idx, 1].set_title(f"Texture Sample {idx}")
    fig.show()

## Convert Axis-Angle Representation to Rotation Matrices

In [ ]:
target_poses_rotmats = batch_rodrigues(target_poses.contiguous().view(-1, 3)).view(-1, 24, 3, 3)
# first entry is global orientation
target_glob_rotmats = target_poses_rotmats[:, 0, :, :]
target_poses_rotmats = target_poses_rotmats[:, 1:, :, :]
if verbosity > 1:
    print(f"Poses rotation matrices: {target_poses_rotmats.shape}")
    print(f"Global rotation matrix: {target_glob_rotmats.shape}")

## Flip Pose Targets such that they are right way up in 3D space

In [ ]:
_, target_glob_rotmats = aa_rotate_rotmats_pytorch3d(
    rotmats=target_glob_rotmats,
    angles=np.pi,
    axes=x_axis,
    rot_mult_order='post'
)

## Compute Target Vertices and Joints

In [ ]:
target_smpl_output = smpl_model(
    body_pose=target_poses_rotmats,
    global_orient=target_glob_rotmats.unsqueeze(1),
    betas=target_shapes,
    pose2rot=False
)
target_vertices = target_smpl_output.vertices
target_joints_all = target_smpl_output.joints
target_joints_h36m = target_joints_all[:, ALL_JOINTS_TO_H36M_MAP, :]
target_joints_h36mlsp = target_joints_h36m[:, H36M_TO_J14, :]
if verbosity > 1:
    print(f"Vertices: {target_vertices.shape}")
    print(f"All joints: {target_joints_all.shape}")
    print(f"Human3.6M joints: {target_joints_h36m.shape}")
    print(f"Human3.6M LSP joints: {target_joints_h36mlsp.shape}")
    
target_reposed_vertices = smpl_model(
    body_pose=torch.zeros_like(target_poses)[:, 3:], # Removes global orientation
    global_orient=torch.zeros_like(target_poses)[:, :3],
    betas=target_shapes    
).vertices
if verbosity > 1:
    print(f"Reposed vertices: {target_reposed_vertices.shape}")

## Prepare Data for Rendering

In [ ]:
target_vertices_for_rendering = aa_rotate_translate_points_pytorch3d(
    points=target_vertices,
    axes=x_axis,
    angles=np.pi,
    translations=torch.zeros(3, device=device).float()
)
target_joints_coco = aa_rotate_translate_points_pytorch3d(
    points=target_joints_all[:, ALL_JOINTS_TO_COCO_MAP, :],
    axes=x_axis,
    angles=np.pi,
    translations=torch.zeros(3, device=device).float()
)
target_joints2d_coco = perspective_project_torch(
    target_joints_coco,
    None,
    target_cam_t,
    focal_length=pose_shape_cfg.TRAIN.SYNTH_DATA.FOCAL_LENGTH,
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)
# Check if joints within image dimensions before cropping + recentering.
target_joints2d_visib_coco = check_joints2d_visibility_torch(
    target_joints2d_coco,
    pose_shape_cfg.DATA.PROXY_REP_SIZE
)
if verbosity > 1:
    print(f"Vertices for rendering: {target_vertices_for_rendering.shape}")
    print(f"COCO joints: {target_joints_coco.shape}")
    print(f"COCO 2D joints: {target_joints2d_coco.shape}")
    print(f"2D joints visible: {target_joints2d_visib_coco.shape}")

## Setup Lighting

In [ ]:
lights_rgb_settings = augment_light(
    batch_size=1,
    device=device,
    rgb_augment_config=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.RGB
)
print(lights_rgb_settings)

## Render

In [ ]:
renderer_output = pytorch3d_renderer(
    vertices=target_vertices_for_rendering,
    textures=textures,
    cam_t=target_cam_t,
    lights_rgb_settings=lights_rgb_settings
)

## Visualize Renders

In [ ]:
if verbosity > 1:
    batch_size = renderer_output['rgb_images'].shape[0]
    fig, axs = plt.subplots(batch_size, 3, figsize=(10,15))
    for idx in range(batch_size):
        axs[idx, 0].imshow(renderer_output['rgb_images'][idx].detach().cpu())
        axs[idx, 0].set_title('RGB')
        axs[idx, 1].imshow(renderer_output['depth_images'][idx].detach().cpu())
        axs[idx, 1].set_title('Depth')
        axs[idx, 2].imshow(renderer_output['iuv_images'][idx].detach().cpu())
        axs[idx, 2].set_title('IUV')
    fig.show()

In [ ]:
import os
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
verbosity = 2
if verbosity > 0:
    print(f"Torch {torch.__version__}")
    if torch.cuda.is_available():
        print("CUDA available")
    else:
        print("CUDA unavailable")
    print(f"Device: {device}")
import sys
sys.path.append('/garmentor')
from data.on_the_fly_smpl_train_dataset import OnTheFlySMPLTrainDataset
from configs import paths
from configs.poseMF_shapeGaussian_net_config import get_poseMF_shapeGaussian_cfg_defaults
from renderers.pytorch3d_textured_renderer import TexturedIUVRenderer
import pytorch3d
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex, TexturesUV
from pytorch3d.io import IO
if verbosity > 0:
    print(f"PyTorch3D {pytorch3d.__version__}")
from models.smpl_official import SMPL
from smplx.lbs import batch_rodrigues
from utils.augmentation.smpl_augmentation import normal_sample_shape
from utils.augmentation.cam_augmentation import augment_cam_t
from utils.augmentation.rgb_augmentation import augment_rgb
from utils.augmentation.lighting_augmentation import augment_light
from utils.rigid_transform_utils import aa_rotate_rotmats_pytorch3d, aa_rotate_translate_points_pytorch3d
import numpy as np
import matplotlib.pyplot as plt

: 

## Load Configuration

In [ ]:
pose_shape_cfg = get_poseMF_shapeGaussian_cfg_defaults()
if verbosity > 1:
    print(f"Configuration: \n{pose_shape_cfg}")

## Some Default Tensors

In [ ]:
x_axis = torch.tensor([1., 0., 0.], device=device, dtype=torch.float32)
delta_betas_std_vector = torch.ones(
    pose_shape_cfg.MODEL.NUM_SMPL_BETAS,
    device=device,
    dtype=torch.float32
) * pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.SMPL.SHAPE_STD
mean_shape = torch.zeros(
    pose_shape_cfg.MODEL.NUM_SMPL_BETAS,
    device=device,
    dtype=torch.float32
)
mean_cam_t = torch.tensor(
    pose_shape_cfg.TRAIN.SYNTH_DATA.MEAN_CAM_T,
    device=device,
    dtype=torch.float32
)
mean_cam_t = mean_cam_t[None, :].expand(pose_shape_cfg.TRAIN.BATCH_SIZE, -1)

## Load Train and Validation Datasets

In [ ]:
train_dataset = OnTheFlySMPLTrainDataset(
    poses_path=paths.TRAIN_POSES_PATH,
    textures_path=paths.TRAIN_TEXTURES_PATH,
    backgrounds_dir_path=paths.TRAIN_BACKGROUNDS_PATH,
    params_from='not_amass',
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)

val_dataset = OnTheFlySMPLTrainDataset(
    poses_path=paths.VAL_POSES_PATH,
    textures_path=paths.VAL_TEXTURES_PATH,
    backgrounds_dir_path=paths.VAL_BACKGROUNDS_PATH,
    params_from='all',
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)
if verbosity > 0:
    print("Training:")
    print(f"    Poses found: {len(train_dataset)}")
    print(f"    Textures found (gray / non-gray): {len(train_dataset.grey_textures)} / {len(train_dataset.nongrey_textures)}")
    print(f"    Backgrounds found: {len(train_dataset.backgrounds_paths)}")
    print("Validation:")
    print(f"    Poses found: {len(val_dataset)}")
    print(f"    Textures found (gray / non-gray): {len(val_dataset.grey_textures)} / {len(val_dataset.nongrey_textures)}")
    print(f"    Backgrounds found: {len(val_dataset.backgrounds_paths)}")

## Train and Validation Data Loaders

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=pose_shape_cfg.TRAIN.NUM_WORKERS,
    pin_memory=pose_shape_cfg.TRAIN.PIN_MEMORY
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=pose_shape_cfg.TRAIN.NUM_WORKERS,
    pin_memory=pose_shape_cfg.TRAIN.PIN_MEMORY
)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}

## Initialize PyTorch3D Renderer

In [ ]:
pytorch3d_renderer = TexturedIUVRenderer(
    device=device,
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE,
    projection_type='perspective',
    perspective_focal_length=pose_shape_cfg.TRAIN.SYNTH_DATA.FOCAL_LENGTH,
    render_rgb=True,
    bin_size=32
)

## Initialize SMPL Model

In [ ]:
smpl_model = SMPL(paths.SMPL, num_betas=pose_shape_cfg.MODEL.NUM_SMPL_BETAS).to(device)

## Fetch Training Data

In [ ]:
for batch in train_dataloader:
    target_poses = batch['pose'].to(device)
    backgrounds = batch['background'].to(device)
    textures = batch['texture'].to(device)
    if verbosity > 0:
        print(f"Poses shape: {target_poses.shape}")
        print(f"Backgrounds shape: {backgrounds.shape}")
        print(f"Textures shape: {textures.shape}")
    break
# Randomly sample body shape
target_shapes = normal_sample_shape(
    batch_size=pose_shape_cfg.TRAIN.BATCH_SIZE,
    mean_shape=mean_shape,
    std_vector=delta_betas_std_vector
)
# Randomly sample camera translation
target_cam_t = augment_cam_t(
    mean_cam_t,
    xy_std=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.CAM.XY_STD,
    delta_z_range=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.CAM.DELTA_Z_RANGE
)
if verbosity > 0:
    print(f"Shapes shape: {target_shapes.shape}")
    print(f"Camera translation shape: {target_cam_t.shape}")

## Visualize the Batch

In [ ]:
if verbosity > 1:
    batch_size = batch['pose'].shape[0]
    fig, axs = plt.subplots(batch_size, 2, figsize=(10,20))
    for idx in range(batch_size):
        axs[idx, 0].imshow(backgrounds[idx].permute(1,2,0).cpu())
        axs[idx, 1].imshow(textures[idx].cpu())
        axs[idx, 0].set_title(f"Background Sample {idx}")
        axs[idx, 1].set_title(f"Texture Sample {idx}")
    fig.show()

## Convert Axis-Angle Representation to Rotation Matrices

In [ ]:
target_poses_rotmats = batch_rodrigues(target_poses.contiguous().view(-1, 3)).view(-1, 24, 3, 3)
# first entry is global orientation
target_glob_rotmats = target_poses_rotmats[:, 0, :, :]
target_poses_rotmats = target_poses_rotmats[:, 1:, :, :]
if verbosity > 1:
    print(f"Poses rotation matrices: {target_poses_rotmats.shape}")
    print(f"Global rotation matrix: {target_glob_rotmats.shape}")

## Flip Pose Targets such that they are right way up in 3D space

In [ ]:
_, target_glob_rotmats = aa_rotate_rotmats_pytorch3d(
    rotmats=target_glob_rotmats,
    angles=np.pi,
    axes=x_axis,
    rot_mult_order='post'
)

## Compute Target Vertices and Joints

In [ ]:
target_smpl_output = smpl_model(
    body_pose=target_poses_rotmats,
    global_orient=target_glob_rotmats.unsqueeze(1),
    betas=target_shapes,
    pose2rot=False
)
target_vertices = target_smpl_output.vertices
target_joints_all = target_smpl_output.joints
target_joints_h36m = target_joints_all[:, ALL_JOINTS_TO_H36M_MAP, :]
target_joints_h36mlsp = target_joints_h36m[:, H36M_TO_J14, :]
if verbosity > 1:
    print(f"Vertices: {target_vertices.shape}")
    print(f"All joints: {target_joints_all.shape}")
    print(f"Human3.6M joints: {target_joints_h36m.shape}")
    print(f"Human3.6M LSP joints: {target_joints_h36mlsp.shape}")
    
target_reposed_vertices = smpl_model(
    body_pose=torch.zeros_like(target_poses)[:, 3:], # Removes global orientation
    global_orient=torch.zeros_like(target_poses)[:, :3],
    betas=target_shapes    
).vertices
if verbosity > 1:
    print(f"Reposed vertices: {target_reposed_vertices.shape}")

## Prepare Data for Rendering

In [ ]:
target_vertices_for_rendering = aa_rotate_translate_points_pytorch3d(
    points=target_vertices,
    axes=x_axis,
    angles=np.pi,
    translations=torch.zeros(3, device=device).float()
)
target_joints_coco = aa_rotate_translate_points_pytorch3d(
    points=target_joints_all[:, ALL_JOINTS_TO_COCO_MAP, :],
    axes=x_axis,
    angles=np.pi,
    translations=torch.zeros(3, device=device).float()
)
target_joints2d_coco = perspective_project_torch(
    target_joints_coco,
    None,
    target_cam_t,
    focal_length=pose_shape_cfg.TRAIN.SYNTH_DATA.FOCAL_LENGTH,
    img_wh=pose_shape_cfg.DATA.PROXY_REP_SIZE
)
# Check if joints within image dimensions before cropping + recentering.
target_joints2d_visib_coco = check_joints2d_visibility_torch(
    target_joints2d_coco,
    pose_shape_cfg.DATA.PROXY_REP_SIZE
)
if verbosity > 1:
    print(f"Vertices for rendering: {target_vertices_for_rendering.shape}")
    print(f"COCO joints: {target_joints_coco.shape}")
    print(f"COCO 2D joints: {target_joints2d_coco.shape}")
    print(f"2D joints visible: {target_joints2d_visib_coco.shape}")

## Setup Lighting

In [ ]:
lights_rgb_settings = augment_light(
    batch_size=1,
    device=device,
    rgb_augment_config=pose_shape_cfg.TRAIN.SYNTH_DATA.AUGMENT.RGB
)
print(lights_rgb_settings)

## Render

In [ ]:
renderer_output = pytorch3d_renderer(
    vertices=target_vertices_for_rendering,
    textures=textures,
    cam_t=target_cam_t,
    lights_rgb_settings=lights_rgb_settings
)

## Visualize Renders

In [ ]:
if verbosity > 1:
    batch_size = renderer_output['rgb_images'].shape[0]
    fig, axs = plt.subplots(batch_size, 3, figsize=(10,15))
    for idx in range(batch_size):
        axs[idx, 0].imshow(renderer_output['rgb_images'][idx].detach().cpu())
        axs[idx, 0].set_title('RGB')
        axs[idx, 1].imshow(renderer_output['depth_images'][idx].detach().cpu())
        axs[idx, 1].set_title('Depth')
        axs[idx, 2].imshow(renderer_output['iuv_images'][idx].detach().cpu())
        axs[idx, 2].set_title('IUV')
    fig.show()